In [1]:
import cv2
import mediapipe as mp
import argparse
from pythonosc import udp_client

import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split

model_save_path = 'model/keypoint_classifier.hdf5'
model = tf.keras.models.load_model(model_save_path)

cap = cv2.VideoCapture(0)
mpHands = mp.solutions.hands
hands = mpHands.Hands()
mpDraw = mp.solutions.drawing_utils

In [2]:
START_SOUND = True
# start the osc
# argparse helps writing user-friendly commandline interfaces
parser = argparse.ArgumentParser()
# OSC server ip: '127.0.0.1'
parser.add_argument("--ip", default='127.0.0.1', help="The ip of the OSC server")
# OSC server port (check on SuperCollider) 57120
parser.add_argument("--port", type=int, default=57120, help="The port the OSC server is listening on")

# Parse the arguments
args, unknown = parser.parse_known_args()

# Start the UDP Client
client = udp_client.SimpleUDPClient(args.ip, args.port)

In [3]:
while True:
    success, image = cap.read()
    image = cv2.flip(image, 1)
    imageRGB = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    results = hands.process(imageRGB)
    center_x = []
    center_y = []
    # checking whether a hand is detected
    if results.multi_hand_landmarks:
        for handLms in results.multi_hand_landmarks: # working with each hand
            for id, lm in enumerate(handLms.landmark):
                h, w, c = image.shape
                cx, cy = int(lm.x * w), int(lm.y * h)
                if(id == 0 or id == 5 or id == 17):
                    center_x.append(cx)
                    center_y.append(cy)
            center_palm_x = int(sum(center_x)/len(center_x))
            center_palm_y = int(sum(center_y)/len(center_y))
            cv2.circle(image, (center_palm_x, center_palm_y), 10, (255, 0 ,255), cv2.FILLED)

            mpDraw.draw_landmarks(image, handLms, mpHands.HAND_CONNECTIONS)

        if START_SOUND:
            client.send_message("freq", center_palm_x)
	    
            client.send_message("amp", center_palm_y)
            
    cv2.imshow("Output", image)
    cv2.waitKey(1)


    
    keypress = cv2.waitKey(1) & 0xFF		
    if keypress == ord("q"):
		    break

# free up memory
cap.release()
cv2.destroyAllWindows()

156
149
162
186
207
208
266
296
315
226
290
318
333
238
298
320
331
245
292
310
319
150
149
166
190
207
207
268
297
316
224
289
317
332
237
297
319
329
245
290
308
317
149
147
164
189
207
208
268
297
316
225
292
320
334
237
298
321
332
245
291
310
319
149
148
165
189
208
205
269
299
318
223
292
320
336
237
299
321
331
245
292
310
318
148
148
167
192
210
212
270
301
321
228
291
320
335
239
297
321
331
246
291
309
318
150
149
167
191
209
208
272
302
320
225
293
321
334
237
299
321
329
245
293
310
318
148
148
167
192
211
210
270
299
316
226
293
320
334
238
300
322
331
244
292
311
319
148
148
166
191
211
211
272
302
320
228
294
321
334
239
300
321
329
246
294
311
319
150
149
167
191
209
208
273
303
322
225
294
322
335
238
300
322
330
245
293
311
319
148
149
167
190
209
209
270
298
315
226
293
320
334
238
299
321
329
245
292
310
317
149
150
168
192
210
209
272
300
317
226
293
320
333
238
298
320
330
245
291
309
317
150
149
166
190
208
211
270
299
316
228
293
319
332
239
299
320
328
246
292


In [34]:
coordinates = np.array([[551, 465], [485, 428], [439,326], [408,307]])
print(coordinates.shape)


(4, 2)


In [35]:
relative_coordinates = np.array(coordinates[0].size, coordinates[1].size)
relative_coordinates.shape

TypeError: Cannot interpret '2' as a data type

In [30]:
relative_coordinates[0,0] = coordinates[0,0] 

IndexError: too many indices for array: array is 1-dimensional, but 2 were indexed

In [27]:
def normalizing_coordinates(coordinates):
    temp1, temp2 = coordinates[0,0], coordinates[0,1]
    relative_coordinates = np.array(coordinates.shape)
    for i in range(coordinates[0].size):
        relative_coordinates[i,0] = coordinates[i,0] - temp1
        relative_coordinates[i,1] = coordinates[i,1] - temp2

    return relative_coordinates

normalized = normalizing_coordinates(coordinates)
print(normalized)

IndexError: too many indices for array: array is 1-dimensional, but 2 were indexed